**River crossing**

*Problem formulation...*

Let $P = \{A,B,C,D\}$ denote the set of individuals, and let
$$
T : P \to \mathbb{R}_+, \quad
T(A)=10,\; T(B)=5,\; T(C)=2,\; T(D)=1
$$
be their respective crossing times in minutes.

**A. States**

A state is defined as a pair
$$
S = (X,\tau),
$$
where:
- $X \subseteq P$ is the set of individuals located on the **left** side of the bridge,
- $\tau \in \{l,r\}$ denotes the position of the torch.

The initial state is
$$
S_0 = (\{A,B,C,D\},\,l)
$$
meaning that all individuals and the torch are on the left side of the bridge.

The terminal state is
$$
S_K = (\varnothing,\,r)
$$
meaning that all individuals have crossed the bridge and the torch is on the right side.


**B. Transitions**

A transition corresponds to a crossing of the bridge and must satisfy the following rules:
- At most 2 individuals can cross simultaneously,
- At least 1 individual must carry the torch while crossing,
- The crossing direction is determined by the position of the torch.

Formally:
- If $\tau = l$, choose a subset $Y \subseteq X$ such that $1 \le |Y| \le 2$,

- If $\tau = r$, choose a subset $Y \subseteq P \notin X$ such that $1 \le |Y| \le 2$.


**C. Crossing time**

The crossing time involving a subset $Y$ is given by
$$
c(Y) = \max_{i \in Y} T(i)
$$


**D. Optimisation problem**

The objective is to find a sequence of admissible states
$$
S_0 \rightarrow S_1 \rightarrow \cdots \rightarrow S_K
$$
minimizing the total crossing time, under the constraints set above:
$$
\sum_{k=1}^{K} c(S_{k-1}, S_k),
$$


In [3]:
import pulp

# Data
people = ["A", "B", "C", "D"]
times = {"A": 10, "B": 5, "C": 2, "D": 1}
K = 5                       
crossings = range(1, K + 1)

# Crossings
forward = [k for k in crossings if k % 2 == 1]
backward = [k for k in crossings if k % 2 == 0]

# -----------------------------
# Problem
# -----------------------------
prob = pulp.LpProblem("River_Crossing", pulp.LpMinimize)

# Variables
x = pulp.LpVariable.dicts(
    "x",
    ((i, k) for i in people for k in crossings),
    cat="Binary"
)

t = pulp.LpVariable.dicts(
    "t",
    crossings,
    lowBound=0,
    cat="Continuous"
)

prob += pulp.lpSum(t[k] for k in crossings)

# -----------------------------
# Constraints
# -----------------------------

# Bridge capacity
for k in crossings:
    prob += pulp.lpSum(x[i, k] for i in people) <= 2
    prob += pulp.lpSum(x[i, k] for i in people) >= 1

# Each individual must cross the bridge
for i in people:
    prob += (
        pulp.lpSum(x[i, k] for k in forward)
        - pulp.lpSum(x[i, k] for k in backward)
        == 1
    )

# Crossing time
for k in crossings:
    for i in people:
        prob += t[k] >= times[i] * x[i, k]


# Solve...
prob.solve(pulp.PULP_CBC_CMD(msg=False))

# Results
print("Status :", pulp.LpStatus[prob.status])
print("Minimum crossing time :", pulp.value(prob.objective), " minutes")

for k in crossings:
    persons = [i for i in people if pulp.value(x[i, k]) > 0.5]
    direction = "→" if k % 2 == 1 else "←"
    print(f"Sequence {k} {direction} : {persons}, time = {pulp.value(t[k])} minute(s)")


Status : Optimal
Minimum crossing time : 17.0  minutes
Sequence 1 → : ['C', 'D'], time = 2.0 minute(s)
Sequence 2 ← : ['D'], time = 1.0 minute(s)
Sequence 3 → : ['C', 'D'], time = 2.0 minute(s)
Sequence 4 ← : ['C'], time = 2.0 minute(s)
Sequence 5 → : ['A', 'B'], time = 10.0 minute(s)
